IBM Certification Project Capstone


In [1]:
import pandas as pd
import numpy  as np

Hello Capstone Project Course!


PART 1 -  Scraping Table from Wiki


In [2]:
from bs4 import BeautifulSoup as bs
import urllib.request

source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

# Create a BeautifulSoup object
soup = bs(source)
#wikitable sortable jquery-tablesorter

tab  = soup.find_all('table')#class_='wikitable sortable jquery-tablesorter')
df = pd.read_html(str(tab))[0]
df[(df['Neighbourhood'] == 'Not assigned') &(df['Borough'] != 'Not assigned')]
df.shape

(180, 3)

In [8]:
df_lat_long = pd.read_csv('/content/Geospatial_Coordinates.csv')
df_lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Part 2 - Adding Lat e Long to Table

In [86]:
df_post_code = pd.merge(df,df_lat_long,how='left',on='Postal Code')
#drop rows with no lat
df_post_code.dropna(axis=0,how='any',inplace=True)

df_post_code.Neighbourhood.value_counts()

Downsview                                                        4
Don Mills                                                        2
Dorset Park, Wexford Heights, Scarborough Town Centre            1
Little Portugal, Trinity                                         1
The Kingsway, Montgomery Road, Old Mill North                    1
                                                                ..
Woburn                                                           1
Canada Post Gateway Processing Centre                            1
Cedarbrae                                                        1
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood    1
The Beaches                                                      1
Name: Neighbourhood, Length: 99, dtype: int64

In [106]:
#keep only one row per neighborhood
df_post_code.Neighbourhood = df_post_code.apply(lambda x : x['Neighbourhood'].split(',')[0], axis=1)


df_post_code = df_post_code.groupby('Neighbourhood').mean().reset_index()

df_post_code = df_post_code[df_post_code.Neighbourhood != 'Business reply mail Processing Centre']
df_post_code = df_post_code[df_post_code.Neighbourhood != 'Canada Post Gateway Processing Centre']

df_post_code.head()

,Neighbourhood,Latitude,Longitude
0,Agincourt,43.794200,-79.262029
1,Alderwood,43.602414,-79.543484
2,Bathurst Manor,43.754328,-79.442259
3,Bayview Village,43.786947,-79.385975
4,Bedford Park,43.733283,-79.419750


<h1>Part 3 - Clustering


**map**

In [11]:
import folium

visualize map of Neighbourhood Toronto


In [13]:
#Toronto geo position
latitude = 43.700373840983396
longitude = -79.37625312910937
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_post_code['Latitude'], df_post_code['Longitude'], df_post_code['Borough'], df_post_code['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


# Connecting to FourSquare with API






In [14]:
CLIENT_ID = 'H00GRYP5I4XZXSQXQ5GLCBNFHOGXX1LWRRUQOPBANTAVG0R0' # your Foursquare ID
CLIENT_SECRET = '1YY5T4ZWLEAWQPNGWN2QIEK1RROH2UWKHZKZLLT4UIPH2ET4' # your Foursquare Secret
ACCESS_TOKEN = 'DTGRP141XRIJKZI031RR4Y231NMGZT2AAQXK02SIUYZMDVYF' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H00GRYP5I4XZXSQXQ5GLCBNFHOGXX1LWRRUQOPBANTAVG0R0
CLIENT_SECRET:1YY5T4ZWLEAWQPNGWN2QIEK1RROH2UWKHZKZLLT4UIPH2ET4


In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [96]:
import requests 
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['groups'][0]['items']


# tranform venues into a dataframe
#dataframe = pd.json_normalize(venues)
#dataframe.head()

nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#venues

In [97]:
import requests 

#Function that extract all venues for each neighbour

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [107]:
latitudes = df_post_code['Latitude']
longitudes = df_post_code['Longitude']
names = df_post_code['Neighbourhood']
Toronto_venues = getNearbyVenues(names, latitudes, longitudes, radius=500)



Agincourt
Alderwood
Bathurst Manor
Bayview Village
Bedford Park
Berczy Park
Birch Cliff
Brockton
CN Tower
Caledonia-Fairbanks
Cedarbrae
Central Bay Street
Christie
Church and Wellesley
Clarks Corners
Cliffside
Commerce Court
Davisville
Davisville North
Del Ray
Don Mills
Dorset Park
Downsview
Dufferin
East Toronto
Eringate
Fairview
First Canadian Place
Forest Hill North & West
Garden District
Glencairn
Golden Mile
Guildwood
Harbourfront East
High Park
Hillcrest Village
Humber Summit
Humberlea
Humewood-Cedarvale
India Bazaar
Islington Avenue
Kennedy Park
Kensington Market
Kingsview Village
Lawrence Manor
Lawrence Park
Leaside
Little Portugal
Malvern
Milliken
Mimico NW
Moore Park
New Toronto
North Park
North Toronto West
Northwest
Northwood Park
Old Mill South
Parkdale
Parkview Hill
Parkwoods
Queen's Park
Regent Park
Richmond
Rosedale
Roselawn
Rouge Hill
Runnymede
Scarborough Village
South Steeles
St. James Town
Steeles West
Stn A PO Boxes
Studio District
Summerhill West
The Annex
The Bea

In [108]:
Toronto_venues.head()
Toronto_venues['Venue Category'].value_counts()

Coffee Shop           180
Café                   86
Restaurant             62
Park                   47
Pizza Place            46
                     ... 
Coworking Space         1
Luggage Store           1
Warehouse Store         1
Airport Food Court      1
Harbor / Marina         1
Name: Venue Category, Length: 271, dtype: int64

In [123]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = ['Neighborhood'] + list(Toronto_onehot.drop('Neighborhood',axis=1).columns)
Toronto_onehot = Toronto_onehot[fixed_columns]

# get frequecy of the category venues in each Neigh
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,...,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alderwood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bathurst Manor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.095238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bedford Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Clothing Store,Drugstore,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
1,Alderwood,Pizza Place,Pharmacy,Sandwich Place,Coffee Shop,Pub,Gym,Airport Lounge,Falafel Restaurant,Ethiopian Restaurant,Escape Room
2,Bathurst Manor,Bank,Coffee Shop,Ice Cream Shop,Diner,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Restaurant,Mobile Phone Shop,Chinese Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
4,Bedford Park,Italian Restaurant,Sandwich Place,Coffee Shop,Locksmith,Indian Restaurant,Restaurant,Sushi Restaurant,Pub,Liquor Store,Café


<h2> K-means

<h5>Unsupervised ML 

In [156]:
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
try:
  neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
except:
  print('insert skipped') 

toronto_merged = df_post_code
toronto_merged.columns =['Neighborhood','Latitude','Longitude']

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

 # check the last columns!
toronto_merged.dropna(how='any',axis=0,inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

insert skipped


,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,43.794200,-79.262029,0,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Clothing Store,Drugstore,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
1,Alderwood,43.602414,-79.543484,0,Pizza Place,Pharmacy,Sandwich Place,Coffee Shop,Pub,Gym,Airport Lounge,Falafel Restaurant,Ethiopian Restaurant,Escape Room
2,Bathurst Manor,43.754328,-79.442259,0,Bank,Coffee Shop,Ice Cream Shop,Diner,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Restaurant,Mobile Phone Shop,Chinese Restaurant
3,Bayview Village,43.786947,-79.385975,0,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
4,Bedford Park,43.733283,-79.419750,0,Italian Restaurant,Sandwich Place,Coffee Shop,Locksmith,Indian Restaurant,Restaurant,Sushi Restaurant,Pub,Liquor Store,Café


In [157]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
#Toronto geo position
latitude = 43.700373840983396
longitude = -79.37625312910937
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [160]:
#Cluster 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.794200,Latin American Restaurant,Breakfast Spot,Skating Rink,Clothing Store,Drugstore,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
1,43.602414,Pharmacy,Sandwich Place,Coffee Shop,Pub,Gym,Airport Lounge,Falafel Restaurant,Ethiopian Restaurant,Escape Room
2,43.754328,Coffee Shop,Ice Cream Shop,Diner,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Restaurant,Mobile Phone Shop,Chinese Restaurant
3,43.786947,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
4,43.733283,Sandwich Place,Coffee Shop,Locksmith,Indian Restaurant,Restaurant,Sushi Restaurant,Pub,Liquor Store,Café
...,...,...,...,...,...,...,...,...,...,...
87,43.750072,Sandwich Place,Bakery,Middle Eastern Restaurant,Auto Garage,Smoke Shop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
88,43.780636,Coffee Shop,Bubble Tea Shop,Burrito Place,Pharmacy,Bank,Food Stand,Grocery Store,Restaurant,Hot Dog Joint
89,43.770992,Mexican Restaurant,Korean BBQ Restaurant,Yoga Studio,Donut Shop,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant
90,43.695344,Park,Beer Store,Athletics & Sports,Dance Studio,Curling Ice,Intersection,Eastern European Restaurant,Dumpling Restaurant,Discount Store


In [162]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,43.689026,Women's Store,Pool,Donut Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
24,43.685347,Park,Convenience Store,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
45,43.728020,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop
49,43.815252,Park,Bakery,Playground,Doner Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run
53,43.713756,Bakery,Construction & Landscaping,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
60,43.753259,Park,Yoga Studio,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
64,43.679563,Trail,Playground,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run
86,43.706876,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Drugstore
92,43.752758,Convenience Store,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant


In [163]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,43.806686,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop


In [164]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,43.724766,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Farmers Market
57,43.636258,Baseball Field,Yoga Studio,Drugstore,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [161]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,43.689574,Trail,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop
68,43.744734,Smoke Shop,Jewelry Store,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run
